#Referenced from https://github.com/NVIDIA-AI-IOT/trt_pose

First, let's load the JSON file which describes the human pose task.  This is in COCO format, it is the category descriptor pulled from the annotations file.  We modify the COCO category slightly, to add a neck keypoint.  We will use this task description JSON to create a topology tensor, which is an intermediate data structure that describes the part linkages, as well as which channels in the part affinity field each linkage corresponds to.

In [ ]:
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

Next, we'll load our model.  Each model takes at least two parameters, *cmap_channels* and *paf_channels* corresponding to the number of heatmap channels
and part affinity field channels.  The number of part affinity field channels is 2x the number of links, because each link has a channel corresponding to the
x and y direction of the vector field for each link.

In [ ]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()

Next, let's load the model weights.  You will need to download these according to the table in the README.

In [ ]:
import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))

In order to optimize with TensorRT using the python library *torch2trt* we'll also need to create some example data.  The dimensions
of this data should match the dimensions that the network was trained with.  Since we're using the resnet18 variant that was trained on
an input resolution of 224x224, we set the width and height to these dimensions.

In [ ]:
WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

Next, we'll use [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) to optimize the model.  We'll enable fp16_mode to allow optimizations to use reduced half precision.

The optimized model may be saved so that we do not need to perform optimization again, we can just load the model.  Please note that TensorRT has device specific optimizations, so you can only use an optimized model on similar platforms.

In [ ]:
import torch2trt

model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)

In [ ]:
OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

We could then load the saved model using *torch2trt* as follows.

In [ ]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

We can benchmark the model in FPS with the following code

In [ ]:
import time

t0 = time.time()
torch.cuda.current_stream().synchronize()
for i in range(50):
    y = model_trt(data)
torch.cuda.current_stream().synchronize()
t1 = time.time()

print(50.0 / (t1 - t0))

Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format.

In [ ]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Next, we'll define two callable classes that will be used to parse the objects from the neural network, as well as draw the parsed objects on an image.

In [ ]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

Assuming you're using NVIDIA Jetson, you can use the [jetcam](https://github.com/NVIDIA-AI-IOT/jetcam) package to create an easy to use camera that will produce images in BGR8/HWC format.

If you're not on Jetson, you may need to adapt the code below.

In [ ]:
from jetcam.usb_camera import USBCamera
# from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
# camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

Connect to the joycontrol tcp server.

Skip the following cell if the joycontrol tcp server is not running yet.

In [ ]:
import asyncio
loop = asyncio.get_event_loop()
tcpReader, tcpWriter = await asyncio.open_connection('127.0.0.1', 8080, loop=loop)

Handler for sending joycontrol command manually.

In [ ]:
def on_button_clicked(b):
    global tcpWriter
    tcpWriter.write(b.tooltip.encode())

ipywidgets for user interactions.

Check the "Control with data" checkbox for sending the game controls from processTrtData to the joycontrol tcp server.

Next, we'll create a widget which will be used to display the camera feed with visualizations.

In [ ]:
import ipywidgets
from IPython.display import display

image_w = ipywidgets.Image(format='jpeg')
display(image_w)

controllerCheckBox_w = ipywidgets.Checkbox(
    value=False,
    description='Control with data',
    disabled=False,
    indent=False
)
display(ipywidgets.HBox([controllerCheckBox_w]))

dataText_w = ipywidgets.Text("Value", description="Data", layout=ipywidgets.Layout(width='100%', height='30px'))
display(dataText_w)

upButton = ipywidgets.Button(tooltip='u',icon='arrow-up')
upButton.on_click(on_button_clicked)
downButton = ipywidgets.Button(tooltip='d',icon='arrow-down')
downButton.on_click(on_button_clicked)
leftButton = ipywidgets.Button(tooltip='l',icon='arrow-left')
leftButton.on_click(on_button_clicked)
rightButton = ipywidgets.Button(tooltip='r',icon='arrow-right')
rightButton.on_click(on_button_clicked)

aButton = ipywidgets.Button(description="A", tooltip='a')
aButton.on_click(on_button_clicked)
bButton = ipywidgets.Button(description="B", tooltip='b')
bButton.on_click(on_button_clicked)
xButton = ipywidgets.Button(description="X", tooltip='x')
xButton.on_click(on_button_clicked)
yButton = ipywidgets.Button(description="Y", tooltip='y')
yButton.on_click(on_button_clicked)
lButton = ipywidgets.Button(description="L", tooltip='L')
lButton.on_click(on_button_clicked)
rButton = ipywidgets.Button(description="R", tooltip='R')
rButton.on_click(on_button_clicked)
lrButton = ipywidgets.Button(description="L+R", tooltip='2')
lrButton.on_click(on_button_clicked)

display(ipywidgets.HBox([upButton, downButton, leftButton, rightButton, aButton, bButton, xButton, yButton, lButton, rButton, lrButton]))


Logging data in batch. For testing.

In [ ]:
from collections import deque
logDataSize = 100
logDataQueue = deque(maxlen=logDataSize)
logDataCounter = 0

def logData(d):
    global logDataQueue
    global logDataCounter
    global dataText_w
    logDataQueue.append(d)
    logDataCounter = logDataCounter + 1
    if logDataCounter == logDataSize:
        logDataCounter = 0
        dataText_w.value = str(logDataQueue)

Major functions for transforming the trt data into game actions. 

In [ ]:
import numpy as np

#flatten the normalized_peaks from torch.Size([1, 18, 100, 2]) to numpy([1, 36 + 1])
#18 peaks * 2 coordinates (y,x) + 1 timestamp
def getFlattenPeaks(object_counts, objects, normalized_peaks, timestamp):
    count = int(object_counts[0])
    flattenPeaks = np.zeros((count,objects[0][0].shape[0]*2 + 1)) #i.e. (1,36 + 1)
    for i in range(count):
        obj = objects[0][i]
        C = obj.shape[0] #18
        flattenPeaks[i][C*2] = timestamp #adding timestamp to the end
        for j in range(C):
            k = int(obj[j]) #obj index, i.e. k=0 for the 1st obj detected
            if k >= 0:
                    peak = normalized_peaks[0][j][k]
                    flattenPeaks[i][j*2] = peak[0]
                    flattenPeaks[i][j*2+1] = peak[1]
    return flattenPeaks

In [ ]:
class MyPose:
    poseIndexNose = 0
    poseIndexLeftEye = 1
    poseIndexRightEye = 2
    poseIndexLeftEar = 3
    poseIndexRightEar = 4
    poseIndexLeftShoulder = 5
    poseIndexRightShoulder = 6
    poseIndexLeftElbow = 7
    poseIndexRightElbow = 8
    poseIndexLeftWrist = 9
    poseIndexRightWrist = 10
    poseIndexLeftHip = 11
    poseIndexRightHip = 12
    poseIndexLeftKnee = 13
    poseIndexRightKnee = 14
    poseIndexLeftAnkle = 15
    poseIndexRightAnkle = 16
    poseIndexNeck = 17
    poseIndexTimeStamp = 18
    flattenIndexTimeStamp = poseIndexTimeStamp * 2

    def xIndexOf(poseIndex):
        return poseIndex * 2 + 1
    
    def yIndexOf(poseIndex):
        return poseIndex * 2
    

In [ ]:
#return absMaxVal, abs(dVSpeed), startingDVSign, isDVInOneSign
def getStatisticsBetweenTwoPeaksInQueue(flattenPeakQueue, flattenIndexA, flattenIndexB):
    absMaxVal = 0
    lastVal = 0
    lastT = 0
    dVSum = 0
    dTSum = 0
    startingDVSign = 0
    isDVInOneSign = True
    lastAbsVal = 0
    isAbsValIncreasing = True
    for i, obj in enumerate(flattenPeakQueue):
        aVal = obj[flattenIndexA]
        bVal = obj[flattenIndexB]
        t = obj[MyPose.flattenIndexTimeStamp]
        val = bVal - aVal
        absVal = abs(val)
        if i == 0:
            absMaxVal = absVal
        else:
            if absVal > absMaxVal: absMaxVal = absVal
            dV = val - lastVal
            dVSign = np.sign(dV)
            if startingDVSign == 0:
                startingDVSign = dVSign
            else:
                if dVSign != 0 and startingDVSign != dVSign: isDVInOneSign = False
            if absVal < lastAbsVal: isAbsValIncreasing = False
            dT = t - lastT
            dVSum = dVSum + dV
            dTSum = dTSum + dT
        lastVal = val
        lastT = t
        lastAbsVal = absVal
    dVSpeed = dVSum / dTSum
    return absMaxVal, abs(dVSpeed), startingDVSign, isDVInOneSign, isAbsValIncreasing

In [ ]:
#TODO: automatically scale these thresholds according to the full body height (i.e. nose to ankle)
isPunchingValThreshold = 0.1
isPunchingSpeedThreshold = 0.05

def isPunchingWithWrist(flattenPeakQueue, shoulderIndex, wristIndex, valThreshold, speedThreshold):
    absMaxVal, absDVSpeed, startingDVSign, isDVInOneSign, isAbsValIncreasing = getStatisticsBetweenTwoPeaksInQueue(flattenPeakQueue, shoulderIndex, wristIndex)
    return absMaxVal > valThreshold and absDVSpeed > speedThreshold and isDVInOneSign and isAbsValIncreasing

def isPunching(flattenPeakQueue):
    global isPunchingValThreshold
    global isPunchingSpeedThreshold
    return isPunchingWithWrist(flattenPeakQueue, MyPose.xIndexOf(MyPose.poseIndexRightShoulder), MyPose.xIndexOf(MyPose.poseIndexRightWrist), isPunchingValThreshold, isPunchingSpeedThreshold) or isPunchingWithWrist(flattenPeakQueue, MyPose.xIndexOf(MyPose.poseIndexLeftShoulder), MyPose.xIndexOf(MyPose.poseIndexLeftWrist), isPunchingValThreshold, isPunchingSpeedThreshold)


In [ ]:
def isPunchingWithWristUpward(flattenPeakQueue, shoulderIndex, wristIndex, valThreshold, speedThreshold):
    absMaxVal, absDVSpeed, startingDVSign, isDVInOneSign, isAbsValIncreasing = getStatisticsBetweenTwoPeaksInQueue(flattenPeakQueue, shoulderIndex, wristIndex)
    return absMaxVal > valThreshold and absDVSpeed > speedThreshold and isDVInOneSign and isAbsValIncreasing and startingDVSign < 0

def isPunchingWithWristDownward(flattenPeakQueue, shoulderIndex, wristIndex, valThreshold, speedThreshold):
    absMaxVal, absDVSpeed, startingDVSign, isDVInOneSign, isAbsValIncreasing = getStatisticsBetweenTwoPeaksInQueue(flattenPeakQueue, shoulderIndex, wristIndex)
    return absMaxVal > valThreshold and absDVSpeed > speedThreshold and isDVInOneSign and isAbsValIncreasing and startingDVSign > 0

def isPunchingUpward(flattenPeakQueue):
    global isPunchingValThreshold
    global isPunchingSpeedThreshold
    return isPunchingWithWristUpward(flattenPeakQueue, MyPose.yIndexOf(MyPose.poseIndexRightShoulder), MyPose.yIndexOf(MyPose.poseIndexRightWrist), isPunchingValThreshold, isPunchingSpeedThreshold) or isPunchingWithWristUpward(flattenPeakQueue, MyPose.yIndexOf(MyPose.poseIndexLeftShoulder), MyPose.yIndexOf(MyPose.poseIndexLeftWrist), isPunchingValThreshold, isPunchingSpeedThreshold)

def isPunchingDownward(flattenPeakQueue):
    global isPunchingValThreshold
    global isPunchingSpeedThreshold
    return isPunchingWithWristDownward(flattenPeakQueue, MyPose.yIndexOf(MyPose.poseIndexRightShoulder), MyPose.yIndexOf(MyPose.poseIndexRightWrist), isPunchingValThreshold, isPunchingSpeedThreshold) or isPunchingWithWristDownward(flattenPeakQueue, MyPose.yIndexOf(MyPose.poseIndexLeftShoulder), MyPose.yIndexOf(MyPose.poseIndexLeftWrist), isPunchingValThreshold, isPunchingSpeedThreshold)


In [ ]:
isKickingValThreshold = 0.04
isKickingSpeedThreshold = 0.01

def isKickingWithAnkle(flattenPeakQueue, hipIndex, ankleIndex, valThreshold, speedThreshold):
    absMaxVal, absDVSpeed, startingDVSign, isDVInOneSign, isAbsValIncreasing = getStatisticsBetweenTwoPeaksInQueue(flattenPeakQueue, hipIndex, ankleIndex)
    return absMaxVal > valThreshold and absDVSpeed > speedThreshold and isDVInOneSign and isAbsValIncreasing
    
def isKicking(flattenPeakQueue):
    global isKickingValThreshold
    global isKickingSpeedThreshold
    return isKickingWithAnkle(flattenPeakQueue, MyPose.xIndexOf(MyPose.poseIndexRightHip), MyPose.xIndexOf(MyPose.poseIndexRightAnkle), isKickingValThreshold, isKickingSpeedThreshold) or isKickingWithAnkle(flattenPeakQueue, MyPose.xIndexOf(MyPose.poseIndexLeftHip), MyPose.xIndexOf(MyPose.poseIndexLeftAnkle), isKickingValThreshold, isKickingSpeedThreshold)


In [ ]:
isBlockingValThreshold = 0.02
isBlockingSpeedThreshold = 0.01

def isBlockingWithWrist(flattenPeakQueue, shoulderIndex, wristIndex, valThreshold, speedThreshold):
    absMaxVal, absDVSpeed, startingDVSign, isDVInOneSign, isAbsValIncreasing = getStatisticsBetweenTwoPeaksInQueue(flattenPeakQueue, shoulderIndex, wristIndex)
    if absMaxVal < valThreshold and absDVSpeed < speedThreshold:
        logData("{} {}".format(absMaxVal, absDVSpeed))
    return absMaxVal < valThreshold and absDVSpeed < speedThreshold

def isBlocking(flattenPeakQueue):
    global isBlockingValThreshold
    global isBlockingSpeedThreshold
    return isBlockingWithWrist(flattenPeakQueue, MyPose.xIndexOf(MyPose.poseIndexRightShoulder), MyPose.xIndexOf(MyPose.poseIndexRightWrist), isBlockingValThreshold, isBlockingSpeedThreshold) and isBlockingWithWrist(flattenPeakQueue, MyPose.xIndexOf(MyPose.poseIndexLeftShoulder), MyPose.xIndexOf(MyPose.poseIndexLeftWrist), isBlockingValThreshold, isBlockingSpeedThreshold)


In [ ]:
def isFlattennPeakQueueValid(flattenPeakQueue):
    obj = flattenPeakQueue[0]
    xVal = obj[MyPose.xIndexOf(MyPose.poseIndexRightWrist)]
    yVal = obj[MyPose.yIndexOf(MyPose.poseIndexRightWrist)]
    return xVal != 0 and yVal != 0


Major game action logic.

In [ ]:
#punch "a"
#kick "x"
#put down "y"
#block "L"
#super "R"
def getKarateControlFrom(flattenPeakQueue):
    if not isFlattennPeakQueueValid(flattenPeakQueue):
        return "N"
    elif isPunching(flattenPeakQueue):
        return "a"
    elif isKicking(flattenPeakQueue):
        return "x"
    elif isPunchingDownward(flattenPeakQueue):
        return "y"
    elif isPunchingUpward(flattenPeakQueue):
        return "R"    
    #elif isBlocking(flattenPeakQueue):
    #    return "L"    #TODO: Throttle the output of L, as joycontrol take 0.1 sec to emulate key pressing
    else:
        return "N" #N for nothing

In [ ]:
lastControlKey = "a"

def setControlKeyButtonStyle(controlKey, style2Set):
    global aButton
    global bButton
    global xButton
    global yButton
    global lButton
    global rButton
    if controlKey == "a":
        aButton.button_style = style2Set
    elif controlKey == "b":
        bButton.button_style = style2Set
    elif controlKey == "x":
        xButton.button_style = style2Set
    elif controlKey == "y":
        yButton.button_style = style2Set
    elif controlKey == "L":
        lButton.button_style = style2Set
    elif controlKey == "R":
        rButton.button_style = style2Set
    
def displayAction(controlKey):
    global lastControlKey
    setControlKeyButtonStyle(lastControlKey, '')
    setControlKeyButtonStyle(controlKey, 'success')
    lastControlKey = controlKey
        

In [ ]:
from collections import deque
N = 5
flattenPeakQueue = deque(maxlen=N)

def processTrtData(object_counts, objects, normalized_peaks, timestamp):
    global N
    global flattenPeakQueue
    global dataText_w
    global controllerCheckBox_w
    global tcpWriter
    flattenPeaks = getFlattenPeaks(object_counts, objects, normalized_peaks, timestamp)
    if len(flattenPeaks) > 0:
        flattenPeak = flattenPeaks[0]
        flattenPeakQueue.append(flattenPeak)
        if len(flattenPeakQueue) == N:
            controlKey = getKarateControlFrom(flattenPeakQueue)
            if controllerCheckBox_w.value and controlKey != "N":
                tcpWriter.write(controlKey.encode())
            displayAction(controlKey)

For measuring the response time of the main processing loop.

In [ ]:
timeSum = 0
timeCount = 0
timeCountN = 1000

def recordTime(deltaTime):
    global timeSum
    global timeCount
    global dataText_w
    global timeCountN
    timeSum = timeSum + deltaTime
    timeCount = timeCount + 1
    if timeCount >= timeCountN:
        fps = timeCountN / timeSum
        timeSum = 0
        timeCount = 0
        dataText_w.value = "fps {}".format(fps)

Finally, we'll define the main execution loop.  This will perform the following steps

1.  Preprocess the camera image
2.  Execute the neural network
3.  Parse the objects from the neural network output
4.  Draw the objects onto the camera image
5.  Convert the image to JPEG format and stream to the display widget

In [ ]:
def execute(change):
    global saveImageCheckBox_w
    global saveImageDelayCounter
    t0 = time.time()
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    processTrtData(counts, objects, peaks, t0)
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    t1 = time.time()
    recordTime(t1-t0)

If we call the cell below it will execute the function once on the current camera frame.

In [ ]:
execute({'new': camera.value})

Call the cell below to attach the execution function to the camera's internal value.  This will cause the execute function to be called whenever a new camera frame is received.

In [ ]:
camera.observe(execute, names='value')

Call the cell below to unattach the camera frame callbacks.

In [ ]:
camera.unobserve_all()